<a href="https://colab.research.google.com/github/TimHBSWFL/UCSD-ML-Capstone/blob/main/CNN_Model_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install torch torchvision transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [18]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import CLIPProcessor, CLIPModel
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
os.listdir('/content/drive/My Drive/Capstone Data Collection/train')

['Fast Food', 'Fine Dining']

In [22]:
train_dir = '/content/drive/My Drive/Capstone Data Collection/train'

In [23]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = ImageFolder(root=train_dir, transform=transform)

In [24]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)

In [25]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

labels = ["fast food", "fine dining"]
text_inputs = processor(text=labels, return_tensors="pt", padding=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPSdpaAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e

In [26]:
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()

# Training loop
epochs = 3
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):

        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model.get_image_features(images)


        print(f"Output shape: {outputs.shape}")

        logits = outputs

        loss = criterion(logits, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 10 == 0:
            print(f"Epoch {epoch+1}, Batch {i}, Loss: {running_loss / (i+1):.4f}")

print("Training complete!")


Output shape: torch.Size([32, 512])
Epoch 1, Batch 0, Loss: 6.1208
Output shape: torch.Size([32, 512])
Output shape: torch.Size([32, 512])
Output shape: torch.Size([32, 512])
Output shape: torch.Size([32, 512])
Output shape: torch.Size([32, 512])
Output shape: torch.Size([32, 512])
Output shape: torch.Size([32, 512])
Output shape: torch.Size([32, 512])
Output shape: torch.Size([32, 512])
Output shape: torch.Size([32, 512])
Epoch 1, Batch 10, Loss: 2.1257
Output shape: torch.Size([32, 512])
Output shape: torch.Size([32, 512])
Output shape: torch.Size([32, 512])
Output shape: torch.Size([32, 512])
Output shape: torch.Size([32, 512])
Output shape: torch.Size([32, 512])
Output shape: torch.Size([32, 512])
Output shape: torch.Size([32, 512])
Output shape: torch.Size([32, 512])
Output shape: torch.Size([32, 512])
Epoch 1, Batch 20, Loss: 1.4563
Output shape: torch.Size([32, 512])
Output shape: torch.Size([32, 512])
Output shape: torch.Size([32, 512])
Output shape: torch.Size([32, 512])
Outpu

In [28]:
model_save_path = "/content/drive/My Drive/Capstone Data Collection/models/yelp_clip_vit_fine_dining_lr5e-5_batch32_2024_09_29_v1.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")


Model saved to /content/drive/My Drive/Capstone Data Collection/models/yelp_clip_vit_fine_dining_lr5e-5_batch32_2024_09_29_v1.pth
